# Exploratory Data Analysis and Clustering on India Census 2011 Dataset

## Project Overview
This comprehensive data science project performs detailed exploratory data analysis (EDA) and clustering on the India Census 2011 dataset. The project aims to:
- Understand demographic patterns across Indian states and districts
- Identify key factors influencing literacy, population growth, and social development
- Apply unsupervised learning techniques to group similar regions
- Provide actionable insights for policy-making and resource allocation

## Dataset Description
The dataset contains demographic information from the 2011 Census of India, including:
- Population statistics (total, male, female, children)
- Literacy rates and educational status
- Work force participation
- Scheduled Caste and Scheduled Tribe populations
- Geographic hierarchies (State, District, Sub-district levels)

---

## 1. Import Required Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.manifold import TSNE

# Statistical analysis (using pandas and numpy instead of scipy)
# Note: Using pandas and numpy functions instead of scipy to avoid memory issues

# File handling
import joblib
import os
import sys

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ All libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🐼 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

## 2. Load and Inspect Dataset

In [ ]:
# Load the dataset
data_path = "../data/2011-IndiaStateDistSbDist-0000.xlsx - Data.csv"
df = pd.read_csv(data_path)

print("📊 Dataset loaded successfully!")
print(f"📏 Dataset shape: {df.shape}")
print(f"📋 Columns: {len(df.columns)}")
print(f"📄 Rows: {len(df)}")

# Display basic information
print("\n" + "="*50)
print("🔍 BASIC DATASET INFORMATION")
print("="*50)

print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Data types:\n{df.dtypes.value_counts()}")

# Display first few rows
print("\n" + "="*50)
print("📋 FIRST 5 ROWS")
print("="*50)
df.head()

In [ ]:
# Detailed information about the dataset
print("🔍 DETAILED DATASET INFORMATION")
print("="*50)
df.info()

print("\n" + "="*50)
print("📊 MISSING VALUES ANALYSIS")
print("="*50)
missing_data = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percentage.values
})

print(f"Total missing values: {df.isnull().sum().sum()}")
print(f"Columns with missing values: {len(missing_df[missing_df['Missing_Count'] > 0])}")

if df.isnull().sum().sum() > 0:
    print("\nColumns with missing values:")
    print(missing_df[missing_df['Missing_Count'] > 0])
else:
    print("✅ No missing values found!")

In [ ]:
# Understand the hierarchical structure
print("🏛️ HIERARCHICAL STRUCTURE ANALYSIS")
print("="*50)

print("Unique values in key columns:")
hierarchical_cols = ['Level', 'TRU']
for col in hierarchical_cols:
    if col in df.columns:
        print(f"{col}: {df[col].unique()}")

# Analyze the administrative levels
if 'Level' in df.columns:
    level_counts = df['Level'].value_counts()
    print(f"\nRecords by administrative level:")
    print(level_counts)

# Analyze Total, Rural, Urban distribution
if 'TRU' in df.columns:
    tru_counts = df['TRU'].value_counts()
    print(f"\nRecords by area type (TRU):")
    print(tru_counts)

# Sample of different levels
print("\n📋 SAMPLE RECORDS BY LEVEL:")
if 'Level' in df.columns:
    for level in df['Level'].unique()[:5]:  # Show first 5 levels
        print(f"\n{level}:")
        sample = df[df['Level'] == level][['State', 'District', 'Name', 'TOT_P', 'P_LIT']].head(3)
        if not sample.empty:
            print(sample)

## 3. Data Cleaning and Preprocessing

In [ ]:
# Create a working copy
df_clean = df.copy()

print("🧹 DATA CLEANING PROCESS")
print("="*50)

# Filter for state-level data only for meaningful analysis
# We'll focus on state-level data for better insights
print("📊 Filtering data for analysis...")

# Keep state-level total records for comprehensive analysis
df_state = df_clean[(df_clean['Level'] == 'STATE') & (df_clean['TRU'] == 'Total')].copy()
print(f"State-level records: {len(df_state)}")

# Also create district-level data for more granular analysis
df_district = df_clean[(df_clean['Level'] == 'DISTRICT') & (df_clean['TRU'] == 'Total')].copy()
print(f"District-level records: {len(df_district)}")

# Remove irrelevant columns for analysis
cols_to_remove = ['State', 'District', 'Subdistt', 'Town/Village', 'Ward', 'EB', 'Level', 'TRU']

# Check if columns exist before removing
existing_cols_to_remove = [col for col in cols_to_remove if col in df_state.columns]
print(f"Removing columns: {existing_cols_to_remove}")

# Keep essential identification columns
df_state_clean = df_state.drop(columns=existing_cols_to_remove, errors='ignore')
df_district_clean = df_district.drop(columns=existing_cols_to_remove, errors='ignore')

# Keep 'Name' column for identification
if 'Name' in df_state.columns:
    df_state_clean['State_Name'] = df_state['Name']
if 'Name' in df_district.columns:
    df_district_clean['District_Name'] = df_district['Name']

print(f"Cleaned state data shape: {df_state_clean.shape}")
print(f"Cleaned district data shape: {df_district_clean.shape}")

# Check for duplicates
print(f"Duplicates in state data: {df_state_clean.duplicated().sum()}")
print(f"Duplicates in district data: {df_district_clean.duplicated().sum()}")

# Remove duplicates if any
df_state_clean = df_state_clean.drop_duplicates()
df_district_clean = df_district_clean.drop_duplicates()

print("✅ Data cleaning completed!")
print(f"Final state data shape: {df_state_clean.shape}")
print(f"Final district data shape: {df_district_clean.shape}")

## 4. Feature Engineering

In [ ]:
# Feature Engineering for State-level data
print("🔧 FEATURE ENGINEERING")
print("="*50)

def create_features(data):
    """Create meaningful features from census data"""
    df_features = data.copy()
    
    # 1. Sex Ratio (Females per 1000 Males)
    if 'TOT_F' in df_features.columns and 'TOT_M' in df_features.columns:
        df_features['Sex_Ratio'] = (df_features['TOT_F'] / df_features['TOT_M']) * 1000
    
    # 2. Literacy Rate (%)
    if 'P_LIT' in df_features.columns and 'TOT_P' in df_features.columns:
        df_features['Literacy_Rate'] = (df_features['P_LIT'] / df_features['TOT_P']) * 100
    
    # 3. Male Literacy Rate (%)
    if 'M_LIT' in df_features.columns and 'TOT_M' in df_features.columns:
        df_features['Male_Literacy_Rate'] = (df_features['M_LIT'] / df_features['TOT_M']) * 100
    
    # 4. Female Literacy Rate (%)
    if 'F_LIT' in df_features.columns and 'TOT_F' in df_features.columns:
        df_features['Female_Literacy_Rate'] = (df_features['F_LIT'] / df_features['TOT_F']) * 100
    
    # 5. Child Population Ratio (0-6 years)
    if 'P_06' in df_features.columns and 'TOT_P' in df_features.columns:
        df_features['Child_Population_Ratio'] = (df_features['P_06'] / df_features['TOT_P']) * 100
    
    # 6. Work Participation Rate
    if 'TOT_WORK_P' in df_features.columns and 'TOT_P' in df_features.columns:
        df_features['Work_Participation_Rate'] = (df_features['TOT_WORK_P'] / df_features['TOT_P']) * 100
    
    # 7. Male Work Participation Rate
    if 'TOT_WORK_M' in df_features.columns and 'TOT_M' in df_features.columns:
        df_features['Male_Work_Rate'] = (df_features['TOT_WORK_M'] / df_features['TOT_M']) * 100
    
    # 8. Female Work Participation Rate
    if 'TOT_WORK_F' in df_features.columns and 'TOT_F' in df_features.columns:
        df_features['Female_Work_Rate'] = (df_features['TOT_WORK_F'] / df_features['TOT_F']) * 100
    
    # 9. SC Population Percentage
    if 'P_SC' in df_features.columns and 'TOT_P' in df_features.columns:
        df_features['SC_Population_Percent'] = (df_features['P_SC'] / df_features['TOT_P']) * 100
    
    # 10. ST Population Percentage
    if 'P_ST' in df_features.columns and 'TOT_P' in df_features.columns:
        df_features['ST_Population_Percent'] = (df_features['P_ST'] / df_features['TOT_P']) * 100
    
    # 11. Dependency Ratio (Non-working population dependency)
    if 'NON_WORK_P' in df_features.columns and 'TOT_WORK_P' in df_features.columns:
        df_features['Dependency_Ratio'] = df_features['NON_WORK_P'] / df_features['TOT_WORK_P']
    
    # 12. Gender Literacy Gap
    if 'Male_Literacy_Rate' in df_features.columns and 'Female_Literacy_Rate' in df_features.columns:
        df_features['Gender_Literacy_Gap'] = df_features['Male_Literacy_Rate'] - df_features['Female_Literacy_Rate']
    
    # 13. Gender Work Gap
    if 'Male_Work_Rate' in df_features.columns and 'Female_Work_Rate' in df_features.columns:
        df_features['Gender_Work_Gap'] = df_features['Male_Work_Rate'] - df_features['Female_Work_Rate']
    
    return df_features

# Apply feature engineering
df_state_features = create_features(df_state_clean)
df_district_features = create_features(df_district_clean)

print("📊 Features created successfully!")
print(f"State data shape after feature engineering: {df_state_features.shape}")
print(f"District data shape after feature engineering: {df_district_features.shape}")

# Display newly created features
new_features = ['Sex_Ratio', 'Literacy_Rate', 'Male_Literacy_Rate', 'Female_Literacy_Rate', 
                'Child_Population_Ratio', 'Work_Participation_Rate', 'Male_Work_Rate', 
                'Female_Work_Rate', 'SC_Population_Percent', 'ST_Population_Percent',
                'Dependency_Ratio', 'Gender_Literacy_Gap', 'Gender_Work_Gap']

existing_new_features = [col for col in new_features if col in df_state_features.columns]
print(f"\nNew features created: {existing_new_features}")

# Display sample of new features
print("\n📋 SAMPLE OF NEW FEATURES (State Level):")
feature_sample = df_state_features[['State_Name'] + existing_new_features].head()
print(feature_sample)

## 5. Exploratory Data Analysis

In [ ]:
# Statistical Summary
print("📊 STATISTICAL SUMMARY - KEY FEATURES")
print("="*70)

# Select key numerical features for analysis
key_features = ['TOT_P', 'Sex_Ratio', 'Literacy_Rate', 'Male_Literacy_Rate', 
                'Female_Literacy_Rate', 'Child_Population_Ratio', 'Work_Participation_Rate',
                'SC_Population_Percent', 'ST_Population_Percent']

# Filter existing features
existing_features = [col for col in key_features if col in df_state_features.columns]

# Statistical summary
stats_summary = df_state_features[existing_features].describe()
print(stats_summary)

# Additional statistics
print("\n📈 ADDITIONAL STATISTICS")
print("="*50)

for feature in existing_features:
    if feature in df_state_features.columns:
        data = df_state_features[feature].dropna()
        print(f"\n{feature}:")
        print(f"  Skewness: {data.skew():.3f}")
        print(f"  Kurtosis: {data.kurtosis():.3f}")
        print(f"  Range: {data.max() - data.min():.2f}")
        print(f"  Coefficient of Variation: {(data.std() / data.mean()) * 100:.2f}%")

### 5.1 Data Visualization

In [ ]:
# Distribution Analysis - Histograms
print("📊 DISTRIBUTION ANALYSIS")
print("="*50)

# Select features for distribution analysis
dist_features = ['Literacy_Rate', 'Sex_Ratio', 'Work_Participation_Rate', 'Child_Population_Ratio']
existing_dist_features = [col for col in dist_features if col in df_state_features.columns]

# Create subplots for distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(existing_dist_features[:4]):
    if feature in df_state_features.columns:
        # Histogram with KDE
        ax = axes[i]
        data = df_state_features[feature].dropna()
        
        ax.hist(data, bins=15, alpha=0.7, color='skyblue', edgecolor='black')
        ax.axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {data.mean():.2f}')
        ax.axvline(data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {data.median():.2f}')
        
        ax.set_title(f'Distribution of {feature}', fontsize=14, fontweight='bold')
        ax.set_xlabel(feature)
        ax.set_ylabel('Frequency')
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Distribution Analysis of Key Features', fontsize=16, fontweight='bold', y=1.02)
plt.show()

# Statistical summary of distributions
print("\n📈 Distribution Statistics:")
for feature in existing_dist_features:
    if feature in df_state_features.columns:
        data = df_state_features[feature].dropna()
        print(f"{feature}: Mean={data.mean():.2f}, Std={data.std():.2f}, Skew={data.skew():.2f}")

In [ ]:
# Box Plots for Outlier Detection
print("📦 OUTLIER DETECTION - BOX PLOTS")
print("="*50)

# Create box plots for key features
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(existing_dist_features[:4]):
    if feature in df_state_features.columns:
        ax = axes[i]
        data = df_state_features[feature].dropna()
        
        bp = ax.boxplot(data, patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        bp['boxes'][0].set_alpha(0.7)
        
        # Calculate and display outliers
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        
        ax.set_title(f'{feature}\\nOutliers: {len(outliers)} ({len(outliers)/len(data)*100:.1f}%)', 
                    fontsize=12, fontweight='bold')
        ax.set_ylabel('Value')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Box Plot Analysis - Outlier Detection', fontsize=16, fontweight='bold', y=1.02)
plt.show()

# Print outlier details
print("\\n📊 OUTLIER SUMMARY:")
for feature in existing_dist_features:
    if feature in df_state_features.columns:
        data = df_state_features[feature].dropna()
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        
        print(f"{feature}: {len(outliers)} outliers ({len(outliers)/len(data)*100:.1f}%)")

In [ ]:
# Correlation Analysis
print("🔗 CORRELATION ANALYSIS")
print("="*50)

# Select numerical features for correlation analysis
correlation_features = ['TOT_P', 'Sex_Ratio', 'Literacy_Rate', 'Male_Literacy_Rate', 
                        'Female_Literacy_Rate', 'Child_Population_Ratio', 'Work_Participation_Rate',
                        'SC_Population_Percent', 'ST_Population_Percent', 'Gender_Literacy_Gap']

# Filter existing features
corr_features = [col for col in correlation_features if col in df_state_features.columns]

# Calculate correlation matrix
correlation_matrix = df_state_features[corr_features].corr()

# Create correlation heatmap
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='RdBu_r', 
            center=0, 
            mask=mask,
            square=True, 
            fmt='.2f',
            cbar_kws={"shrink": 0.8})

plt.title('Correlation Heatmap - Census Features', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print highest correlations
print("\\n🔍 HIGHEST CORRELATIONS:")
# Get upper triangle of correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find highest correlations
high_corr = []
for col in upper_triangle.columns:
    for row in upper_triangle.index:
        corr_val = upper_triangle.loc[row, col]
        if not pd.isna(corr_val) and abs(corr_val) > 0.5:
            high_corr.append((row, col, corr_val))

# Sort by absolute correlation value
high_corr.sort(key=lambda x: abs(x[2]), reverse=True)

for i, (var1, var2, corr) in enumerate(high_corr[:10]):  # Show top 10
    print(f"{i+1:2d}. {var1} <-> {var2}: {corr:.3f}")

In [ ]:
# Bivariate Analysis - Scatter Plots
print("📈 BIVARIATE ANALYSIS - SCATTER PLOTS")
print("="*50)

# Create interesting scatter plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Literacy Rate vs Sex Ratio
ax1 = axes[0, 0]
if 'Literacy_Rate' in df_state_features.columns and 'Sex_Ratio' in df_state_features.columns:
    scatter = ax1.scatter(df_state_features['Sex_Ratio'], 
                         df_state_features['Literacy_Rate'],
                         c=df_state_features['TOT_P'], 
                         cmap='viridis', 
                         alpha=0.7, 
                         s=100)
    ax1.set_xlabel('Sex Ratio (Females per 1000 Males)')
    ax1.set_ylabel('Literacy Rate (%)')
    ax1.set_title('Literacy Rate vs Sex Ratio\\n(Color: Population Size)', fontweight='bold')
    plt.colorbar(scatter, ax=ax1, label='Total Population')
    ax1.grid(True, alpha=0.3)

# 2. Work Participation vs Literacy Rate
ax2 = axes[0, 1]
if 'Work_Participation_Rate' in df_state_features.columns and 'Literacy_Rate' in df_state_features.columns:
    ax2.scatter(df_state_features['Literacy_Rate'], 
               df_state_features['Work_Participation_Rate'],
               alpha=0.7, 
               color='coral', 
               s=100)
    ax2.set_xlabel('Literacy Rate (%)')
    ax2.set_ylabel('Work Participation Rate (%)')
    ax2.set_title('Work Participation vs Literacy Rate', fontweight='bold')
    ax2.grid(True, alpha=0.3)

# 3. Gender Literacy Gap vs Overall Literacy
ax3 = axes[1, 0]
if 'Gender_Literacy_Gap' in df_state_features.columns and 'Literacy_Rate' in df_state_features.columns:
    ax3.scatter(df_state_features['Literacy_Rate'], 
               df_state_features['Gender_Literacy_Gap'],
               alpha=0.7, 
               color='lightgreen', 
               s=100)
    ax3.set_xlabel('Overall Literacy Rate (%)')
    ax3.set_ylabel('Gender Literacy Gap (Male - Female %)')
    ax3.set_title('Gender Literacy Gap vs Overall Literacy', fontweight='bold')
    ax3.grid(True, alpha=0.3)

# 4. SC Population vs Literacy Rate
ax4 = axes[1, 1]
if 'SC_Population_Percent' in df_state_features.columns and 'Literacy_Rate' in df_state_features.columns:
    ax4.scatter(df_state_features['SC_Population_Percent'], 
               df_state_features['Literacy_Rate'],
               alpha=0.7, 
               color='orange', 
               s=100)
    ax4.set_xlabel('SC Population (%)')
    ax4.set_ylabel('Literacy Rate (%)')
    ax4.set_title('Literacy Rate vs SC Population %', fontweight='bold')
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Bivariate Analysis - Key Relationships', fontsize=16, fontweight='bold', y=1.02)
plt.show()

# Print correlation coefficients for the relationships
print("\\n📊 CORRELATION COEFFICIENTS:")
relationships = [
    ('Sex_Ratio', 'Literacy_Rate'),
    ('Literacy_Rate', 'Work_Participation_Rate'),
    ('Literacy_Rate', 'Gender_Literacy_Gap'),
    ('SC_Population_Percent', 'Literacy_Rate')
]

for var1, var2 in relationships:
    if var1 in df_state_features.columns and var2 in df_state_features.columns:
        corr = df_state_features[var1].corr(df_state_features[var2])
        print(f"{var1} vs {var2}: {corr:.3f}")

## 6. Clustering Analysis (Unsupervised Learning)

In [ ]:
# Feature Selection for Clustering
print("🎯 FEATURE SELECTION FOR CLUSTERING")
print("="*50)

# Select key features for clustering analysis
clustering_features = [
    'Literacy_Rate', 'Sex_Ratio', 'Work_Participation_Rate', 
    'Child_Population_Ratio', 'SC_Population_Percent', 'ST_Population_Percent',
    'Gender_Literacy_Gap', 'TOT_P'
]

# Filter existing features
available_features = [col for col in clustering_features if col in df_state_features.columns]
print(f"Available features for clustering: {available_features}")

# Create dataset for clustering
cluster_data = df_state_features[available_features + ['State_Name']].copy()

# Remove any rows with missing values
cluster_data_clean = cluster_data.dropna()
print(f"\\nData shape for clustering: {cluster_data_clean.shape}")
print(f"States included in clustering: {len(cluster_data_clean)}")

# Prepare features (exclude State_Name)
X = cluster_data_clean[available_features].copy()

# Display basic statistics of clustering features
print("\\n📊 CLUSTERING FEATURES STATISTICS:")
print(X.describe().round(2))

In [ ]:
# Data Standardization
print("⚖️ DATA STANDARDIZATION")
print("="*50)

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

print("✅ Features standardized successfully!")
print(f"Scaled data shape: {X_scaled_df.shape}")

# Display before and after scaling comparison
print("\\n📊 BEFORE AND AFTER SCALING COMPARISON:")
print("\\nBefore scaling (first 5 rows):")
print(X.head().round(2))

print("\\nAfter scaling (first 5 rows):")
print(X_scaled_df.head().round(2))

# Verify standardization
print("\\n🔍 STANDARDIZATION VERIFICATION:")
print("Mean values after scaling (should be ~0):")
print(X_scaled_df.mean().round(6))
print("\\nStandard deviation after scaling (should be ~1):")
print(X_scaled_df.std().round(6))

In [ ]:
# Determine Optimal Number of Clusters
print("🔍 DETERMINING OPTIMAL NUMBER OF CLUSTERS")
print("="*60)

# Range of clusters to test
k_range = range(2, 11)
inertias = []
silhouette_scores = []

print("Testing different numbers of clusters...")

# Calculate metrics for different k values
for k in k_range:
    # KMeans clustering
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # Calculate inertia (within-cluster sum of squares)
    inertias.append(kmeans.inertia_)
    
    # Calculate silhouette score
    sil_score = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(sil_score)
    
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette Score={sil_score:.3f}")

# Plot Elbow Method and Silhouette Score
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Elbow Method Plot
ax1.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Within-cluster Sum of Squares)')
ax1.set_title('Elbow Method for Optimal k', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Add annotations for key points
for i, (k, inertia) in enumerate(zip(k_range, inertias)):
    ax1.annotate(f'{inertia:.1f}', (k, inertia), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=10)

# Silhouette Score Plot
ax2.plot(k_range, silhouette_scores, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score for Different k', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add annotations for silhouette scores
for i, (k, score) in enumerate(zip(k_range, silhouette_scores)):
    ax2.annotate(f'{score:.3f}', (k, score), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# Find optimal k
optimal_k_silhouette = k_range[np.argmax(silhouette_scores)]
max_silhouette = max(silhouette_scores)

print(f"\\n🎯 OPTIMAL NUMBER OF CLUSTERS:")
print(f"Based on Silhouette Score: k = {optimal_k_silhouette} (Score: {max_silhouette:.3f})")

# Calculate elbow point (simple method - looking for the point where the rate of decrease slows)
# Calculate the differences
diffs = [inertias[i] - inertias[i+1] for i in range(len(inertias)-1)]
# Find where the difference starts to level off
elbow_k = k_range[np.argmax(np.diff(diffs))] + 2  # Add 2 to adjust for indexing
print(f"Based on Elbow Method: k ≈ {elbow_k}")

# Use the silhouette score optimal k for final clustering
final_k = optimal_k_silhouette
print(f"\\n✅ Selected k = {final_k} for final clustering")

In [ ]:
# KMeans Clustering Implementation
print("🎯 K-MEANS CLUSTERING IMPLEMENTATION")
print("="*50)

# Initialize and fit KMeans with optimal k
kmeans_final = KMeans(n_clusters=final_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled)

# Add cluster labels to the original data
cluster_data_clean['Cluster'] = cluster_labels

print(f"✅ KMeans clustering completed with k={final_k}")
print(f"Final silhouette score: {silhouette_score(X_scaled, cluster_labels):.3f}")

# Cluster distribution
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
print(f"\\n📊 CLUSTER DISTRIBUTION:")
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count} states ({count/len(cluster_labels)*100:.1f}%)")

# Display states in each cluster
print(f"\\n🏛️ STATES BY CLUSTER:")
for cluster_id in sorted(cluster_data_clean['Cluster'].unique()):
    states_in_cluster = cluster_data_clean[cluster_data_clean['Cluster'] == cluster_id]['State_Name'].tolist()
    print(f"\\nCluster {cluster_id} ({len(states_in_cluster)} states):")
    print(", ".join(states_in_cluster))

In [ ]:
# Principal Component Analysis for Visualization
print("📊 PRINCIPAL COMPONENT ANALYSIS FOR VISUALIZATION")
print("="*60)

# Apply PCA for 2D visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print(f"✅ PCA completed")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.3f} ({pca.explained_variance_ratio_.sum()*100:.1f}%)")

# Create PCA DataFrame
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=cluster_data_clean.index)
pca_df['Cluster'] = cluster_labels
pca_df['State_Name'] = cluster_data_clean['State_Name']

# Visualize clusters in 2D PCA space
plt.figure(figsize=(14, 10))

# Create scatter plot
colors = plt.cm.Set1(np.linspace(0, 1, final_k))
for i in range(final_k):
    cluster_mask = pca_df['Cluster'] == i
    plt.scatter(pca_df[cluster_mask]['PC1'], 
               pca_df[cluster_mask]['PC2'],
               c=[colors[i]], 
               label=f'Cluster {i}', 
               s=100, 
               alpha=0.7,
               edgecolors='black',
               linewidth=0.5)

# Add state labels to points
for idx, row in pca_df.iterrows():
    plt.annotate(row['State_Name'], 
                (row['PC1'], row['PC2']),
                xytext=(5, 5), 
                textcoords='offset points',
                fontsize=9,
                alpha=0.8)

# Add cluster centers
centers_pca = pca.transform(kmeans_final.cluster_centers_)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], 
           c='red', marker='x', s=200, linewidths=3, label='Centroids')

plt.xlabel(f'First Principal Component ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'Second Principal Component ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('K-Means Clustering Results - PCA Visualization\\n(India Census 2011 State-level Data)', 
         fontweight='bold', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print PCA component interpretation
print(f"\\n🔍 PCA COMPONENTS INTERPRETATION:")
feature_names = X.columns
components_df = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=feature_names
)

print("\\nComponent loadings (how much each feature contributes to each component):")
print(components_df.round(3))

# Find features with highest absolute loadings for each component
print("\\n📈 MAIN DRIVERS OF EACH COMPONENT:")
for i, pc in enumerate(['PC1', 'PC2']):
    loadings = components_df[pc].abs().sort_values(ascending=False)
    print(f"\\n{pc} (explains {pca.explained_variance_ratio_[i]:.1%} of variance):")
    for j, (feature, loading) in enumerate(loadings.head(3).items()):
        direction = "+" if components_df.loc[feature, pc] > 0 else "-"
        print(f"  {j+1}. {feature}: {direction}{loading:.3f}")

## 7. Cluster Analysis and Interpretation

In [ ]:
# Cluster Characteristics Analysis
print("📊 CLUSTER CHARACTERISTICS ANALYSIS")
print("="*60)

# Calculate mean values for each cluster
cluster_means = cluster_data_clean.groupby('Cluster')[available_features].mean()

print("📈 CLUSTER MEAN VALUES:")
print(cluster_means.round(2))

# Create a comprehensive comparison table
cluster_summary = []
for cluster_id in sorted(cluster_data_clean['Cluster'].unique()):
    cluster_subset = cluster_data_clean[cluster_data_clean['Cluster'] == cluster_id]
    
    summary = {
        'Cluster': cluster_id,
        'States_Count': len(cluster_subset),
        'Avg_Literacy_Rate': cluster_subset['Literacy_Rate'].mean(),
        'Avg_Sex_Ratio': cluster_subset['Sex_Ratio'].mean(),
        'Avg_Work_Participation': cluster_subset['Work_Participation_Rate'].mean(),
        'Avg_Child_Population': cluster_subset['Child_Population_Ratio'].mean(),
        'Avg_SC_Population': cluster_subset['SC_Population_Percent'].mean(),
        'Avg_ST_Population': cluster_subset['ST_Population_Percent'].mean(),
        'Avg_Population': cluster_subset['TOT_P'].mean()
    }
    cluster_summary.append(summary)

summary_df = pd.DataFrame(cluster_summary)
print(f"\\n📋 CLUSTER SUMMARY TABLE:")
print(summary_df.round(2))

# Visualize cluster characteristics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Literacy Rate by Cluster
ax1 = axes[0, 0]
cluster_means['Literacy_Rate'].plot(kind='bar', ax=ax1, color='skyblue', alpha=0.8)
ax1.set_title('Average Literacy Rate by Cluster', fontweight='bold')
ax1.set_ylabel('Literacy Rate (%)')
ax1.set_xlabel('Cluster')
ax1.tick_params(axis='x', rotation=0)
ax1.grid(True, alpha=0.3)

# 2. Sex Ratio by Cluster
ax2 = axes[0, 1]
cluster_means['Sex_Ratio'].plot(kind='bar', ax=ax2, color='lightcoral', alpha=0.8)
ax2.set_title('Average Sex Ratio by Cluster', fontweight='bold')
ax2.set_ylabel('Sex Ratio (F per 1000 M)')
ax2.set_xlabel('Cluster')
ax2.tick_params(axis='x', rotation=0)
ax2.grid(True, alpha=0.3)

# 3. Work Participation by Cluster
ax3 = axes[1, 0]
cluster_means['Work_Participation_Rate'].plot(kind='bar', ax=ax3, color='lightgreen', alpha=0.8)
ax3.set_title('Average Work Participation Rate by Cluster', fontweight='bold')
ax3.set_ylabel('Work Participation (%)')
ax3.set_xlabel('Cluster')
ax3.tick_params(axis='x', rotation=0)
ax3.grid(True, alpha=0.3)

# 4. Population Size by Cluster
ax4 = axes[1, 1]
cluster_means['TOT_P'].plot(kind='bar', ax=ax4, color='orange', alpha=0.8)
ax4.set_title('Average Population Size by Cluster', fontweight='bold')
ax4.set_ylabel('Population (in millions)')
ax4.set_xlabel('Cluster')
ax4.tick_params(axis='x', rotation=0)
ax4.grid(True, alpha=0.3)

# Format population values to millions
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))

plt.tight_layout()
plt.suptitle('Cluster Characteristics Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.show()

In [ ]:
# Detailed Cluster Interpretation
print("🔍 DETAILED CLUSTER INTERPRETATION")
print("="*60)

# Analyze each cluster in detail
for cluster_id in sorted(cluster_data_clean['Cluster'].unique()):
    cluster_subset = cluster_data_clean[cluster_data_clean['Cluster'] == cluster_id]
    
    print(f"\\n{'='*20} CLUSTER {cluster_id} {'='*20}")
    print(f"Number of States: {len(cluster_subset)}")
    print(f"States: {', '.join(cluster_subset['State_Name'].tolist())}")
    
    # Key characteristics
    print(f"\\n📊 Key Characteristics:")
    print(f"  • Literacy Rate: {cluster_subset['Literacy_Rate'].mean():.1f}% (Range: {cluster_subset['Literacy_Rate'].min():.1f}% - {cluster_subset['Literacy_Rate'].max():.1f}%)")
    print(f"  • Sex Ratio: {cluster_subset['Sex_Ratio'].mean():.0f} (Range: {cluster_subset['Sex_Ratio'].min():.0f} - {cluster_subset['Sex_Ratio'].max():.0f})")
    print(f"  • Work Participation: {cluster_subset['Work_Participation_Rate'].mean():.1f}% (Range: {cluster_subset['Work_Participation_Rate'].min():.1f}% - {cluster_subset['Work_Participation_Rate'].max():.1f}%)")
    print(f"  • Child Population: {cluster_subset['Child_Population_Ratio'].mean():.1f}% (Range: {cluster_subset['Child_Population_Ratio'].min():.1f}% - {cluster_subset['Child_Population_Ratio'].max():.1f}%)")
    print(f"  • SC Population: {cluster_subset['SC_Population_Percent'].mean():.1f}% (Range: {cluster_subset['SC_Population_Percent'].min():.1f}% - {cluster_subset['SC_Population_Percent'].max():.1f}%)")
    print(f"  • ST Population: {cluster_subset['ST_Population_Percent'].mean():.1f}% (Range: {cluster_subset['ST_Population_Percent'].min():.1f}% - {cluster_subset['ST_Population_Percent'].max():.1f}%)")
    print(f"  • Average Population: {cluster_subset['TOT_P'].mean()/1e6:.1f} million")
    
    if 'Gender_Literacy_Gap' in cluster_subset.columns:
        print(f"  • Gender Literacy Gap: {cluster_subset['Gender_Literacy_Gap'].mean():.1f} percentage points")

# Provide interpretive labels for clusters based on characteristics
print(f"\\n🏷️ CLUSTER INTERPRETATION & LABELING")
print("="*60)

cluster_interpretations = {}

for cluster_id in sorted(cluster_data_clean['Cluster'].unique()):
    cluster_subset = cluster_data_clean[cluster_data_clean['Cluster'] == cluster_id]
    
    # Determine cluster characteristics
    literacy = cluster_subset['Literacy_Rate'].mean()
    sex_ratio = cluster_subset['Sex_Ratio'].mean()
    work_participation = cluster_subset['Work_Participation_Rate'].mean()
    st_population = cluster_subset['ST_Population_Percent'].mean()
    
    # Generate interpretation based on characteristics
    if literacy > 85:
        literacy_label = "High Literacy"
    elif literacy > 70:
        literacy_label = "Moderate Literacy"
    else:
        literacy_label = "Low Literacy"
    
    if sex_ratio > 950:
        gender_label = "Good Gender Balance"
    elif sex_ratio > 900:
        gender_label = "Moderate Gender Balance"
    else:
        gender_label = "Poor Gender Balance"
    
    if work_participation > 45:
        work_label = "High Work Participation"
    elif work_participation > 35:
        work_label = "Moderate Work Participation"
    else:
        work_label = "Low Work Participation"
    
    if st_population > 20:
        tribal_label = "High Tribal Population"
    elif st_population > 10:
        tribal_label = "Moderate Tribal Population"
    else:
        tribal_label = "Low Tribal Population"
    
    # Combine characteristics for cluster label
    interpretation = f"{literacy_label}, {work_label}, {tribal_label}"
    cluster_interpretations[cluster_id] = interpretation
    
    print(f"\\nCluster {cluster_id}: {interpretation}")
    print(f"  Profile: {literacy_label} ({literacy:.1f}%), {work_label} ({work_participation:.1f}%), {tribal_label} ({st_population:.1f}%)")

# Add interpretative labels to the data
cluster_data_clean['Cluster_Label'] = cluster_data_clean['Cluster'].map(cluster_interpretations)

print(f"\\n✅ Cluster analysis completed!")
print(f"Summary: {len(cluster_data_clean)} states grouped into {final_k} distinct clusters based on demographic characteristics.")

## 8. Save Processed Data and Models

In [ ]:
# Save processed data and models
print("💾 SAVING PROCESSED DATA AND MODELS")
print("="*50)

# Create outputs directory if it doesn't exist
output_dir = "../outputs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✅ Created output directory: {output_dir}")

# 1. Save cleaned dataset with features
cleaned_data_path = os.path.join(output_dir, "cleaned_dataset.csv")
df_state_features.to_csv(cleaned_data_path, index=False)
print(f"✅ Saved cleaned dataset: {cleaned_data_path}")

# 2. Save cluster results
cluster_results_path = os.path.join(output_dir, "cluster_results.csv")
cluster_data_clean.to_csv(cluster_results_path, index=False)
print(f"✅ Saved cluster results: {cluster_results_path}")

# 3. Save the trained KMeans model
model_path = os.path.join(output_dir, "kmeans_model.pkl")
joblib.dump(kmeans_final, model_path)
print(f"✅ Saved KMeans model: {model_path}")

# 4. Save the scaler
scaler_path = os.path.join(output_dir, "scaler.pkl")
joblib.dump(scaler, scaler_path)
print(f"✅ Saved StandardScaler: {scaler_path}")

# 5. Save PCA model
pca_path = os.path.join(output_dir, "pca_model.pkl")
joblib.dump(pca, pca_path)
print(f"✅ Saved PCA model: {pca_path}")

# 6. Save feature names used in clustering
features_path = os.path.join(output_dir, "feature_names.txt")
with open(features_path, 'w') as f:
    for feature in available_features:
        f.write(f"{feature}\\n")
print(f"✅ Saved feature names: {features_path}")

# 7. Create a summary report
summary_path = os.path.join(output_dir, "analysis_summary.txt")
with open(summary_path, 'w') as f:
    f.write("INDIA CENSUS 2011 EDA & CLUSTERING ANALYSIS SUMMARY\\n")
    f.write("="*60 + "\\n\\n")
    f.write(f"Dataset: India Census 2011\\n")
    f.write(f"Analysis Level: State-level data\\n")
    f.write(f"Total States Analyzed: {len(cluster_data_clean)}\\n")
    f.write(f"Features Used for Clustering: {len(available_features)}\\n")
    f.write(f"Optimal Number of Clusters: {final_k}\\n")
    f.write(f"Final Silhouette Score: {silhouette_score(X_scaled, cluster_labels):.3f}\\n\\n")
    
    f.write("CLUSTER INTERPRETATIONS:\\n")
    f.write("-" * 30 + "\\n")
    for cluster_id, interpretation in cluster_interpretations.items():
        states_in_cluster = cluster_data_clean[cluster_data_clean['Cluster'] == cluster_id]['State_Name'].tolist()
        f.write(f"\\nCluster {cluster_id}: {interpretation}\\n")
        f.write(f"States ({len(states_in_cluster)}): {', '.join(states_in_cluster)}\\n")
    
    f.write(f"\\n\\nFEATURES USED:\\n")
    f.write("-" * 15 + "\\n")
    for i, feature in enumerate(available_features, 1):
        f.write(f"{i}. {feature}\\n")

print(f"✅ Saved analysis summary: {summary_path}")

# Display saved files
print(f"\\n📁 FILES SAVED TO {output_dir}:")
saved_files = [
    "cleaned_dataset.csv",
    "cluster_results.csv", 
    "kmeans_model.pkl",
    "scaler.pkl",
    "pca_model.pkl",
    "feature_names.txt",
    "analysis_summary.txt"m 
]

for file in saved_files:
    file_path = os.path.join(output_dir, file)
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        print(f"  ✓ {file} ({file_size:,} bytes)")

print(f"\\n🎉 Analysis completed successfully!")
print(f"📊 {len(cluster_data_clean)} states clustered into {final_k} groups")
print(f"💾 All results saved to {output_dir}")
print(f"\\n📋 Next steps:")
print(f"  1. Review cluster results in cluster_results.csv")
print(f"  2. Use saved models for predictions on new data")
print(f"  3. Run the Streamlit dashboard for interactive visualization")
print(f"  4. Command: streamlit run ../dashboard/app.py")